In [1]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
f1 = "C:\\Users\\kjhoe\\OneDrive\\바탕화~1-LAPTOP-VGOF9FJ7-25904429\\대외활동\\공모전\\2024 빅콘\\개체번호, 연도 통합.xlsx"
df= pd.read_excel(f1, sheet_name='combined')

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17544 entries, 0 to 17543
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   FARM_ID          17544 non-null  object        
 1   MSRM_DT          17544 non-null  datetime64[ns]
 2   AREA_TPRT        17539 non-null  float64       
 3   AREA_HMDT        17517 non-null  float64       
 4   AREA_WDSP        17544 non-null  float64       
 5   INNER_TPRT       10100 non-null  float64       
 6   INNER_HMDT       9615 non-null   float64       
 7   INNER_CBDX_CTRN  7528 non-null   float64       
 8   EXTN_TPRT        0 non-null      float64       
 9   PLT_LNGTH        120 non-null    float64       
 10  LEAF_LNGTH       1488 non-null   float64       
 11  LEAF_WDTH        1488 non-null   float64       
 12  STEM_THNS        1488 non-null   float64       
 13  PLANT_AR_SQM     17544 non-null  int64         
 14  SHPMN_QTY        2640 non-null   float

In [5]:
# 날짜 컬럼을 datetime 형식으로 변환
df['MSRM_DT'] = pd.to_datetime(df['MSRM_DT'])


In [6]:
# 각 컬럼의 결측치 비율 확인
missing_values = df.isnull().sum() / len(df) * 100
print("결측치 비율:\n", missing_values)


결측치 비율:
 FARM_ID              0.000000
MSRM_DT              0.000000
AREA_TPRT            0.028500
AREA_HMDT            0.153899
AREA_WDSP            0.000000
INNER_TPRT          42.430461
INNER_HMDT          45.194938
INNER_CBDX_CTRN     57.090743
EXTN_TPRT          100.000000
PLT_LNGTH           99.316005
LEAF_LNGTH          91.518468
LEAF_WDTH           91.518468
STEM_THNS           91.518468
PLANT_AR_SQM         0.000000
SHPMN_QTY           84.952120
SHPMN_AMT           84.952120
dtype: float64


In [7]:
#MICE 이용하여 지역, 내부환경 결측값 채우기

In [8]:
df2 = df.copy(deep=True)

In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17544 entries, 0 to 17543
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   FARM_ID          17544 non-null  object        
 1   MSRM_DT          17544 non-null  datetime64[ns]
 2   AREA_TPRT        17539 non-null  float64       
 3   AREA_HMDT        17517 non-null  float64       
 4   AREA_WDSP        17544 non-null  float64       
 5   INNER_TPRT       10100 non-null  float64       
 6   INNER_HMDT       9615 non-null   float64       
 7   INNER_CBDX_CTRN  7528 non-null   float64       
 8   EXTN_TPRT        0 non-null      float64       
 9   PLT_LNGTH        120 non-null    float64       
 10  LEAF_LNGTH       1488 non-null   float64       
 11  LEAF_WDTH        1488 non-null   float64       
 12  STEM_THNS        1488 non-null   float64       
 13  PLANT_AR_SQM     17544 non-null  int64         
 14  SHPMN_QTY        2640 non-null   float

In [46]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error
import random

# 결측값을 MICE 모델로 채우는 함수
def fill_missing_with_mice(df, max_iter=100, random_state=0):
    # IterativeImputer 초기화
    imputer = IterativeImputer(max_iter=max_iter, random_state=random_state)

    # 전체 데이터프레임에서 결측값 대체
    imputed_data = imputer.fit_transform(df)

    # 원래의 데이터프레임과 동일한 구조로 데이터프레임 생성
    imputed_df = pd.DataFrame(imputed_data, columns=df.columns, index=df.index)

    return imputed_df

# 임의로 결측값 추가하는 함수
def introduce_missing_values_randomly(df, missing_fraction=0.2, random_state=0):
    np.random.seed(random_state)
    df_with_missing = df.copy()

    # 총 셀 수에서 결측값을 추가할 개수 계산
    total_cells = df.size
    num_missing = int(total_cells * missing_fraction)

    # 랜덤하게 결측치를 추가할 위치 선택
    for _ in range(num_missing):
        row = random.randint(0, df.shape[0] - 1)
        col = random.randint(0, df.shape[1] - 1)
        df_with_missing.iat[row, col] = np.nan

    return df_with_missing

# 환경 변수 리스트 (사용자가 제공한 변수)
env_vars = ['AREA_TPRT', 'AREA_HMDT', 'INNER_TPRT', 'INNER_HMDT', 'INNER_CBDX_CTRN']

# 결측치가 없는 행만 필터링
df_no_missing = df2[env_vars].dropna()

# 결측치 없는 데이터에서 일부 값을 임의로 결측치로 변환
df_with_random_missing = introduce_missing_values_randomly(df_no_missing, missing_fraction=0.3)

# MICE로 결측치를 채움
df_imputed = fill_missing_with_mice(df_with_random_missing)
# 결측치 없는 데이터의 인덱스를 원본 데이터에서 추출
selected_indices = df_no_missing.index

# 원본 데이터 df2에서 필터링된 인덱스만 뽑아 새로운 df3 생성
df3 = df2.loc[selected_indices]

def evaluate_performance(original_series, imputed_series):

    original_values = original_series
    imputed_values = imputed_series

    mse = mean_squared_error(original_values, imputed_values)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(original_values, imputed_values)

    # 평가 결과 출력
    print(f'MSE: {mse}')
    print(f'RMSE: {rmse}')
    print(f'MAE: {mae}')

# 각 변수에 대해 성능 평가
for var in env_vars:
    print(f"Evaluating performance for {var}...")

    # 성능 평가 (임의로 결측치로 만든 값들에 대해 원래 값과 대체된 값 비교)
    evaluate_performance(df3[var], df_imputed[var])


Evaluating performance for AREA_TPRT...
MSE: 10.50634397927865
RMSE: 3.24134909864375
MAE: 1.28538396131081
Evaluating performance for AREA_HMDT...
MSE: 117.36257883754323
RMSE: 10.833401074341484
MAE: 4.367617277745367
Evaluating performance for INNER_TPRT...
MSE: 6.3165672538162045
RMSE: 2.513278188704188
MAE: 0.9929431520089682
Evaluating performance for INNER_HMDT...
MSE: 51.733554156781956
RMSE: 7.192604129019055
MAE: 2.85891968967636
Evaluating performance for INNER_CBDX_CTRN...
MSE: 13247.83113916402
RMSE: 115.09922301720381
MAE: 45.89280280765964


In [47]:
df_imputed

,AREA_TPRT,AREA_HMDT,INNER_TPRT,INNER_HMDT,INNER_CBDX_CTRN
180,3.400000,54.406624,22.1,72.000000,765.0
181,4.100000,33.000000,34.3,87.000000,601.0
182,4.200000,28.000000,40.6,61.000000,985.0
183,4.400000,24.000000,35.7,28.946526,868.0
184,3.500000,29.000000,27.8,58.000000,813.0
...,...,...,...,...,...
14241,28.100000,94.000000,24.8,95.300000,632.0
14242,22.074922,78.000000,25.0,96.700000,858.0
14243,18.543066,71.000000,26.1,69.891969,944.0
14244,32.100000,64.000000,28.2,89.500000,937.0


In [48]:
df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7295 entries, 180 to 14255
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   AREA_TPRT        7295 non-null   float64
 1   AREA_HMDT        7295 non-null   float64
 2   INNER_TPRT       7295 non-null   float64
 3   INNER_HMDT       7295 non-null   float64
 4   INNER_CBDX_CTRN  7295 non-null   float64
dtypes: float64(5)
memory usage: 342.0 KB


In [49]:
df4 = df.copy(deep=True)

In [50]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer  # Required to enable the IterativeImputer
from sklearn.impute import IterativeImputer

# Defining the environmental variables
env_vars = ['AREA_TPRT', 'AREA_HMDT', 'INNER_TPRT', 'INNER_HMDT', 'INNER_CBDX_CTRN']

# Applying MICE using IterativeImputer only to the selected columns
imputer = IterativeImputer(max_iter=10, random_state=0)

# Fit the imputer on env_vars and transform the data
imputed_values = imputer.fit_transform(df2[env_vars])

# Update the original DataFrame with the imputed values for the specified columns
df4[env_vars] = imputed_values

# Results
print("Data After Imputing Missing Values for Selected Columns with MICE:")
print(df4.head())

Data After Imputing Missing Values for Selected Columns with MICE:
              FARM_ID             MSRM_DT  AREA_TPRT  AREA_HMDT  AREA_WDSP  \
0  GBE_KRMLN_002000at 2019-01-01 00:00:00       -1.9       50.0        1.6   
1  GBE_KRMLN_002000at 2019-01-01 01:00:00       -3.2       54.0        1.7   
2  GBE_KRMLN_002000at 2019-01-01 02:00:00       -2.5       62.0        2.1   
3  GBE_KRMLN_002000at 2019-01-01 03:00:00       -2.5       62.0        1.6   
4  GBE_KRMLN_002000at 2019-01-01 04:00:00       -3.6       67.0        0.8   

   INNER_TPRT  INNER_HMDT  INNER_CBDX_CTRN  EXTN_TPRT  PLT_LNGTH  LEAF_LNGTH  \
0        19.1        87.0       840.911874        NaN        NaN         NaN   
1        18.8        87.0       856.601990        NaN        NaN         NaN   
2        18.5        87.0       871.298626        NaN        NaN         NaN   
3        18.1        87.0       878.309889        NaN        NaN         NaN   
4        17.8        88.0       893.520363        NaN        NaN

In [51]:
# 각 컬럼의 결측치 비율 확인
missing_values = df4.isnull().sum() / len(df) * 100
print("결측치 비율:\n", missing_values)

결측치 비율:
 FARM_ID              0.000000
MSRM_DT              0.000000
AREA_TPRT            0.000000
AREA_HMDT            0.000000
AREA_WDSP            0.000000
INNER_TPRT           0.000000
INNER_HMDT           0.000000
INNER_CBDX_CTRN      0.000000
EXTN_TPRT          100.000000
PLT_LNGTH           99.316005
LEAF_LNGTH          91.518468
LEAF_WDTH           91.518468
STEM_THNS           91.518468
PLANT_AR_SQM         0.000000
SHPMN_QTY           84.952120
SHPMN_AMT           84.952120
dtype: float64
